# Launch Sites Locations Analysis with Folium

In [1]:
# Import libraries
import folium
import wget
import pandas as pd

ModuleNotFoundError: No module named 'folium'

In [ ]:
# Folium Plugins
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

## Task 1: Mark all launch sites on a map

In [ ]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

In [ ]:
spacex_df.head()

In [ ]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df.head()

In [ ]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [ ]:
# Create a blue circle at NASA JPC's coordinate with a pop label
# showing its name
circle = folium.Circle(nasa_coordinate, 
                       radius=1000, 
                       color='#d35400', 
                       fill=True).add_child(folium.Popup('Nasa Johnson Space Center'))
# Create a blue cirle at NASA JPC's coordinate with a icon showing
# its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d3540;"><b>%s</b></div>' % 'NASA JSC',
    )
)
site_map.add_child(circle)
site_map.add_child(marker)

In [ ]:
launch_sites_df.head()

In [ ]:
def add_marker_cirle(lat, long, name):
    item_coordinate = [lat, long]
    # Create a blue circle 
    circle = folium.Circle(item_coordinate, 
                       radius=100, 
                       color='#d35400', 
                       fill=True).add_child(folium.Popup(name))
    marker = folium.map.Marker(
        item_coordinate,
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d3540;"><b>%s</b></div>' % name,
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

In [ ]:
# Inital the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=2)
# For each launch site, add a circle object based on its coordinate (lat, long)
# In addition, add Launch site name as a popup label
for index in range(launch_sites_df.shape[0]):
    item = launch_sites_df.iloc[index,:]
#     print(item['Lat'])
#     print(item['Long'])
#     print(item['Launch Site'])
    add_marker_cirle(item['Lat'], 
                     item['Long'],
                     item['Launch Site']
                    )
    
        

In [ ]:
site_map

## Task 2: Mark the success/failed launches for each site on the map

In [ ]:
spacex_df.tail(10)

In [ ]:
marker_cluster = MarkerCluster()

In [ ]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

In [ ]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker = folium.Marker(
        [
            record['Lat'],
            record['Long']
        ],
        # Create an icon as a text label
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map

## Task 3: Calculate the distances between a launch site to its proximities

In [ ]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse
# over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)
site_map

In [ ]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [ ]:
# Find coordinate of the closet coastline
coastline_lat = 28.56359
coastline_long = -80.56798
distances = []
for index, item in spacex_df.iterrows():
    launch_site_lat = item['Lat']
    launch_site_long = item['Long']
    distance = calculate_distance(launch_site_lat, launch_site_long,
                                  coastline_lat, coastline_long)
    distances.append(distance)
distance_min = min(distances)
print('distance_min=',distance_min)

In [ ]:
spacex_grouped_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
spacex_grouped_df

In [ ]:
distance_marker = folium.Marker(
    [coastline_lat, coastline_long],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_min)
    )
)

site_map.add_child(distance_marker)
site_map

In [ ]:
line = folium.PolyLine(locations=[[coastline_lat, coastline_long], [28.563197, -80.576820]], weight=1)
site_map.add_child(line)